# IBM contest 2019

Let's look at a "simple" example to illustrate the workings of **tweedledum** oracle compilation infrastructure. The examples is called *the Zed city problem*, which is an instance of vertex coloring.  He I will first introduce the problem and then show how to apply Grover's search to solve it while using advance oracle compilations techniques.

Vertex coloring is the problem of assigning colors to vertices of a graph such that adjacent vertices are not of the same color. Our example is a variation of this problem presented in the final challenge of [IBM's virtual hackathon](https://github.com/quantum-challenge/2019/): 

*Zed city is a newly established (fictitious) municipality in Tokyo composed of $11$ districts.  Four convenience store chains $A$, $B$, $C$, and $D$ have each built their first store in this new city.  The goal is to use vertex coloring to distribute stores in the districts that still do not have one yet, while ensuring that there is only one store per district and that adjacent districts do not have stores from the same chain. The following figure shows Zed city as an undirected graph.*

<img src="../images/zed_city.png" alt="Quantum teleportation" style="width: 400px;"/>

We start by creating Boolean modelling for the problem. We assign to each vertex in the graph a binary string that represents a color and formulate the following constraints: 
 1. every vertex must have one color assigned to it
 2. two adjacent vertices cannot have the same color

Let's define and create an encoding for each of the colors:

In [1]:
from tweedledum.bool_function_compiler import BitVec

orange = BitVec('00')
blue = BitVec('01')
yellow = BitVec('10')
green = BitVec('11')

# In the contest this enconding was given to the participants

Now we need an oracle, i.e., a Boolean function `f(v0, ..., v6)` that evaluates to $1$ only for those variable assignment that represent a graph coloring satisfying all constraints.

## Oracles

In [2]:
from tweedledum.bool_function_compiler import BitVec

def naive(v0, v1, v2, v3, v4, v5, v6 : BitVec(2)) -> BitVec(1):
    c0 = (v0 != BitVec('00'))
    c1 = (v1 != BitVec('01')) and (v1 != v0)
    c2 = (v2 != BitVec('00')) and (v2 != BitVec('10')) and (v2 != v0)
    c3 = (v3 != BitVec('00')) and (v3 != v0) and (v3 != v1) and (v3 != v2)
    c4 = (v4 != BitVec('01')) and (v4 != v1) and (v4 != v3)
    c5 = (v5 != BitVec('11')) and (v5 != v2) and (v5 != v3)
    c6 = (v6 != BitVec('11')) and (v6 != v2) and (v6 != v3) and (v6 != v4) and (v6 != v5)
    return c0 and c1 and c2 and c3 and c4 and c5 and c6

Classically, we can solve a vertex coloring problem that is model as such by querying `f` with all input combinations until we find one for which `f` evaluates to one. Hence, in the worst case, we will query the oracle $O(2^{n})$ times. However, a quantum computer can solve such a problem with high probability by querying $f$ only $O(\sqrt{2^n})$.

Let's try some inputs:

In [3]:
print(f"Try 1: {naive(orange, blue, yellow, green, orange, blue, green)}")
print(f"Try 2: {naive(yellow, blue, yellow, green, green, blue, green)}")
print(f"Try 3: {naive(yellow, green, blue, orange, green, yellow, orange)}")
# This gets tiring quite fast

Try 1: 0
Try 2: 0
Try 3: 0


In [4]:
# Using l337 programming skills:
num_solutions = 0
whole_search_space = 2**(2*7)
for i in range(2**(2*7)):
    v = BitVec(14, i)
    result = naive(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12])
    if result:
        num_solutions += 1
        print(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12])

print(f"Search space size: {whole_search_space}")
print(f"Number of solutions: {num_solutions}")

01 00 11 10 11 01 00
10 00 11 01 10 10 00
10 11 11 01 10 10 00
10 00 01 11 10 10 00
10 00 11 01 11 10 00
01 11 11 10 00 00 01
01 00 11 10 11 00 01
10 11 11 01 00 00 10
10 00 11 01 11 00 10
Search space size: 16384
Number of solutions: 9


Seems legit! After brute forcing my way to the solutions, I find the same 9 solutions provided as reference after the constest ended. 

Okay, the fucntion is doing what is supposed to do! We could go on and build our quantum circuit using this function but we can do better than this. In this particular problem instance we know before hand that some assigments (or combination of assigments) of the variable are inconsistant i.e., violate one or more of the constraints, and cannot participate in any solution.

For example, `v0` is adjacent to `A` which is colored with orange. Hence we don't need to try any assigment that will place orange on `v0`. If we promisse to not try this, we can simplify `f`!! Indeed, all those comparisons against constants can go away:

In [5]:
def naive_opt(v0, v1, v2, v3, v4, v5, v6 : BitVec(2)) -> BitVec(1):
    c1 = (v1 != v0)
    c2 = (v2 != v0)
    c3 = (v3 != v0) and (v3 != v1) and (v3 != v2)
    c4 = (v4 != v1) and (v4 != v3)
    c5 = (v5 != v2) and (v5 != v3)
    c6 = (v6 != v2) and (v6 != v3) and (v6 != v4) and (v6 != v5)
    return c1 and c2 and c3 and c4 and c5 and c6

# I go further with other hand optmizations
# (not really important for this discussion)
def fancy(v0, v1, v2, v3, v4, v5, v6 : BitVec(2)) -> BitVec(1):
    c1 = (v1[0] == v1[1]) and (v3 != v1)
    c023 = ((v0 ^ v2 ^ v3) == BitVec('00'))
    c4 = (v4 != v1) and (v4 != v3)
    c5 = (v5 != v2) and (v5 != v3)
    c6 = ((v2 ^ v3 ^ v5 ^ v6) == BitVec('00')) and (v6 != v4)
    return c1 and c023 and c4 and c5 and c6

We can represent the set of inputs which we promise not to use as a function:

In [6]:
def dont_care(v0, v1, v2, v3, v4, v5, v6 : BitVec(2)) -> BitVec(1):
    return ((v0 == BitVec('00')) or (v1 == BitVec('01')) or (v2 == BitVec('00')) or
            (v2 == BitVec('10')) or (v3 == BitVec('00')) or (v4 == BitVec('01')) or
            (v5 == BitVec('11')) or (v6 == BitVec('11')))

In [7]:
# Using l337 programming skills:
pruned_search_space = 0

# Loop through all 2^14 possible inputs
for i in range(2**(2*7)):
    v = BitVec(14, i)
    # Check if is the ones we promise to not use:
    if dont_care(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12]):
        continue
    pruned_search_space += 1
    result = fancy(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12])
    if result:
        print(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12])

print(f"Pruned search space size: {pruned_search_space}")
print(f"Number of solutions: {num_solutions}")

01 00 11 10 11 01 00
10 00 11 01 10 10 00
10 11 11 01 10 10 00
10 00 01 11 10 10 00
10 00 11 01 11 10 00
01 11 11 10 00 00 01
01 00 11 10 11 00 01
10 11 11 01 00 00 10
10 00 11 01 11 00 10
Pruned search space size: 1458
Number of solutions: 9


## Configurations

In [8]:
oracle_type = 'fancy' # (naive, fancy)
syntehsis_method = 'xag' # (pkrm, xag)

# Optimization options
use_barenco_decomp = True
use_linear_resynth = True
use_diagonal_synth = True
use_xag_optimization = True
use_gate_cancellation = True

## Concrete implementation

In [9]:
from tweedledum.bool_function_compiler import BitVec, BoolFunction

oracle_func = BoolFunction(naive) if oracle_type == 'naive' else BoolFunction(fancy)

In [10]:
import numpy as np
search_space_size = whole_search_space if oracle_type == 'naive' else pruned_search_space
num_iterations = int(np.floor(np.sqrt(search_space_size / num_solutions)))

print(f"Search space size: {search_space_size}")
print(f"Number of solutions: {num_solutions}")
print(f"Grover iterations: {num_iterations} (theory)")

Search space size: 1458
Number of solutions: 9
Grover iterations: 12 (theory)


In [11]:
from tweedledum.synthesis import xag_synth, pkrm_synth

def use_pkrm(bool_function):
    return pkrm_synth(bool_function.truth_table(output_bit=0))

def use_xag(bool_function):
    from tweedledum.passes import parity_decomp, linear_resynth
    if use_xag_optimization:
        from tweedledum.classical import optimize
        optimize(bool_function.logic_network())
    circuit = xag_synth(bool_function.logic_network())
    if use_linear_resynth:
        circuit = linear_resynth(circuit)
    return parity_decomp(circuit)

In [12]:
oracle_circuit = use_pkrm(oracle_func) if syntehsis_method == 'pkrm' else use_xag(oracle_func)
if use_barenco_decomp:
    from tweedledum.passes import barenco_decomp
    oracle_circuit = barenco_decomp(oracle_circuit, {'max_qubits' : 32})
print(f"Number of qubits: {oracle_circuit.num_qubits()}")
print(f"Number of instructions: {len(oracle_circuit)}")

Number of qubits: 30
Number of instructions: 119


## Initialization subcircuit

In [13]:
from tweedledum.ir import Circuit
from tweedledum.operators import H, Ry, X
from tweedledum.passes import inverse

def naive_init():
    circuit = Circuit()
    qubits = [circuit.create_qubit() for i in range(14)]
    for qubit in qubits:
        circuit.apply_operator(H(), [qubit])
    return circuit
        
def fancy_init():
    theta = 2 * np.arccos(1 / np.sqrt(3))
    circuit = Circuit()
    qubits = [circuit.create_qubit() for i in range(14)]
    #district0
    circuit.apply_operator(Ry(theta), [qubits[0]])
    circuit.apply_operator(H(), [qubits[0], qubits[1]])
    circuit.apply_operator(X(), [qubits[1]])

    #district1
    circuit.apply_operator(Ry(theta), [qubits[2]])
    circuit.apply_operator(H(), [qubits[2], qubits[3]])

    #district2
    circuit.apply_operator(H(), [qubits[4]])
    circuit.apply_operator(X(), [qubits[5]])

    #district3
    circuit.apply_operator(Ry(theta), [qubits[6]])
    circuit.apply_operator(H(), [qubits[6], qubits[7]])
    circuit.apply_operator(X(), [qubits[7]])

    #district4 B
    circuit.apply_operator(Ry(theta), [qubits[8]])
    circuit.apply_operator(H(), [qubits[8], qubits[9]])

    #district5
    circuit.apply_operator(Ry(theta), [qubits[10]])
    circuit.apply_operator(H(), [qubits[10], qubits[11]])
    circuit.apply_operator(X(), [qubits[10]])

    #district6
    circuit.apply_operator(Ry(theta), [qubits[12]])
    circuit.apply_operator(H(), [qubits[12], qubits[13]])
    circuit.apply_operator(X(), [qubits[12]])
        
    return circuit

init_subcircuit = naive_init() if oracle_type == 'naive' else fancy_init()
init_adj_subcircuit = inverse(init_subcircuit)

## Diffuser subcircuit

In [14]:
from tweedledum.operators import X, Z, Measure

diffuser_subcircuit = Circuit()
qubits = [diffuser_subcircuit.create_qubit() for i in range(15)] 
# Why 15? I need to account for the output qubit! Also if barenco is used
# this qubit should be left alone as it where the output rests!

diffuser_subcircuit.append(init_adj_subcircuit, qubits[0:init_subcircuit.num_qubits()], [])
for qubit in qubits[0:14]:
    diffuser_subcircuit.apply_operator(X(), [qubit])
diffuser_subcircuit.apply_operator(Z(), qubits[0:14])
for qubit in qubits[0:14]:
    diffuser_subcircuit.apply_operator(X(), [qubit])
diffuser_subcircuit.append(init_subcircuit, qubits[0:init_adj_subcircuit.num_qubits()], [])

if use_barenco_decomp:
    from tweedledum.passes import barenco_decomp
    diffuser_subcircuit = barenco_decomp(diffuser_subcircuit, {'max_qubits' : 32})

## Grover circuit

In [15]:
# Initialize
num_qubits = max(oracle_circuit.num_qubits(), diffuser_subcircuit.num_qubits())
circuit = Circuit()
qubits = [circuit.create_qubit() for i in range(num_qubits)]
cbits = [circuit.create_cbit() for i in range(14)]

circuit.apply_operator(X(), [qubits[14]])
circuit.apply_operator(H(), [qubits[14]])

circuit.append(init_subcircuit, qubits[0:init_subcircuit.num_qubits()], [])

# Grover iteration
num_iterations = num_iterations if oracle_type == 'naive' else 5
# I hard code the number of iteration to 5 when using 'fancy' oracles 
# because the contest said it was okay to do so. Howeverm, in theory,
# this number should be sqrt(1458/9) ~ 12
for i in range(num_iterations):
    circuit.append(oracle_circuit, qubits[0:oracle_circuit.num_qubits()], [])
    circuit.append(diffuser_subcircuit, qubits[0:diffuser_subcircuit.num_qubits()], [])

if use_diagonal_synth:
    from tweedledum.ir import rotation_angle
    from tweedledum.passes import shallow_duplicate
    from tweedledum.synthesis import diagonal_synth

    new_circuit = shallow_duplicate(circuit)
    for instruction in circuit:
        if instruction.kind() == 'std.rx':
            angle = rotation_angle(instruction)
            qs = instruction.qubits()
            angles = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -angle/2, angle/2]
            new_circuit.apply_operator(H(), [qs[-1]])
            diagonal_synth(new_circuit, qs, instruction.cbits(), angles)
            new_circuit.apply_operator(H(), [qs[-1]])
        else:
            new_circuit.apply_operator(instruction)
    circuit = new_circuit

for i in range(circuit.num_cbits()):
    circuit.apply_operator(Measure(), [qubits[i]], [cbits[i]])

print(f"Number of qubits: {circuit.num_qubits()}")
print(f"Number of instructions: {len(circuit)}")

Number of qubits: 30
Number of instructions: 3404


In [16]:
if use_gate_cancellation:
    from tweedledum.passes import gate_cancellation
    circuit = gate_cancellation(circuit)
    
print(f"Number of qubits: {circuit.num_qubits()}")
print(f"Number of instructions: {len(circuit)}")

Number of qubits: 30
Number of instructions: 3144


## Translate to QISKIT

In [17]:
from tweedledum.converters_qiskit import tweedledum_to_qiskit_qc

qiskit_circuit = tweedledum_to_qiskit_qc(circuit)

## Local simulation

In [23]:
import qiskit
from qiskit.providers.aer import QasmSimulator

# Construct an ideal simulator
sim = QasmSimulator()

# Perform an ideal simulation
result = qiskit.execute(qiskit_circuit, sim).result()
counts = result.get_counts(0)

In [24]:
# Sort count
count_sorted = sorted(counts.items(), key=lambda x:x[1], reverse=True)

for bit_string, count in count_sorted:
    v = BitVec(bit_string)
    print(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12], f'({count})', (oracle_func.simulate(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12])))

10 00 11 01 11 00 10 (105) 1
10 00 01 11 10 10 00 (98) 1
10 11 11 01 00 00 10 (96) 1
10 11 11 01 10 10 00 (96) 1
01 11 11 10 00 00 01 (95) 1
10 00 11 01 11 10 00 (93) 1
10 00 11 01 10 10 00 (92) 1
01 00 11 10 11 00 01 (89) 1
01 00 11 10 11 01 00 (88) 1
10 10 11 10 01 00 01 (1) 0
01 00 11 01 10 00 01 (1) 0
10 00 11 10 10 00 01 (1) 0
01 00 10 11 10 00 01 (1) 0
01 10 11 11 10 00 01 (1) 0
01 10 11 00 11 00 01 (1) 0
11 00 00 01 11 00 01 (1) 0
00 11 00 01 11 00 01 (1) 0
11 11 00 10 00 01 01 (1) 0
10 10 11 00 01 01 01 (1) 0
01 10 10 10 01 01 01 (1) 0
11 11 11 10 01 01 01 (1) 0
00 11 10 11 10 01 01 (1) 0
10 11 11 00 11 01 01 (1) 0
11 11 11 01 01 00 00 (1) 0
00 10 01 11 00 10 01 (1) 0
01 01 11 11 00 10 01 (1) 0
10 01 10 00 01 10 01 (1) 0
00 11 00 10 01 10 01 (1) 0
11 10 10 00 10 10 01 (1) 0
01 11 10 00 10 10 01 (1) 0
10 11 00 00 11 10 01 (1) 0
01 10 11 01 11 10 01 (1) 0
10 00 01 10 00 11 01 (1) 0
00 11 00 11 00 11 01 (1) 0
00 11 01 11 01 11 01 (1) 0
00 00 01 00 11 11 01 (1) 0
00 01 00 10 11 11 

## Online simulation

In [18]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
pass_ = Unroller(['u', 'cx'])
pm = PassManager(pass_)
qiskit_circuit = pm.run(qiskit_circuit) 

gates = qiskit_circuit.count_ops()
cost = gates['u'] + (10 * gates['cx'])
print(cost)

19065


In [19]:
from qiskit import IBMQ

provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')
job = backend.run(qiskit_circuit, shots=8000, seed_simulator=12345, fusion_enable=True)
result = job.result()
counts = result.get_counts()

In [20]:
# Sort count
count_sorted = sorted(counts.items(), key=lambda x:x[1], reverse=True)

for bit_string, count in count_sorted:
    v = BitVec(bit_string)
    print(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12], f'({count})', (oracle_func.simulate(v[2:0], v[4:2], v[6:4], v[8:6], v[10:8], v[12:10], v[14:12])))

10 00 11 01 11 00 10 (560) 1
10 00 11 01 11 10 00 (543) 1
01 11 11 10 01 01 00 (523) 1
01 00 11 10 11 01 00 (520) 1
01 00 11 10 01 01 00 (519) 1
01 11 11 10 00 00 01 (518) 1
01 00 10 11 01 01 00 (506) 1
01 00 11 10 11 00 01 (494) 1
10 11 11 01 00 00 10 (468) 1
11 11 11 11 00 01 01 (8) 0
01 00 11 01 01 01 01 (8) 0
10 01 11 10 11 10 10 (8) 0
11 00 10 10 01 01 00 (8) 0
11 11 10 11 00 10 00 (8) 0
10 01 11 11 00 10 00 (8) 0
10 00 11 01 01 00 00 (7) 0
01 01 11 01 01 00 10 (7) 0
01 00 11 11 01 00 10 (7) 0
01 01 10 11 11 00 10 (7) 0
01 11 11 01 11 10 10 (7) 0
01 01 11 01 00 10 00 (7) 0
01 01 11 01 11 10 00 (7) 0
10 01 10 01 00 00 01 (6) 0
01 11 10 01 00 00 01 (6) 0
10 11 10 01 00 00 01 (6) 0
11 11 10 11 00 00 01 (6) 0
11 01 11 10 11 00 01 (6) 0
01 01 10 01 00 01 01 (6) 0
01 00 10 10 00 01 01 (6) 0
11 01 10 10 00 01 01 (6) 0
01 01 11 01 01 01 01 (6) 0
01 00 10 01 01 00 00 (6) 0
01 01 11 10 00 10 01 (6) 0
10 11 10 01 01 10 01 (6) 0
11 00 11 10 11 10 01 (6) 0
01 00 11 01 11 00 10 (6) 0
10 11 11 1

01 01 11 01 01 00 00 (3) 0
01 11 10 01 11 01 01 (3) 0
11 11 10 01 11 01 01 (3) 0
11 01 11 01 01 00 00 (3) 0
11 01 11 01 11 01 01 (3) 0
10 00 10 10 11 01 01 (3) 0
01 01 10 10 11 01 01 (3) 0
11 11 10 10 11 01 01 (3) 0
10 11 11 10 11 01 01 (3) 0
11 11 11 10 11 01 01 (3) 0
11 00 11 11 11 01 01 (3) 0
10 11 11 11 11 01 01 (3) 0
01 01 10 01 00 10 01 (3) 0
10 01 10 01 00 10 01 (3) 0
01 11 10 01 00 10 01 (3) 0
11 11 10 01 00 10 01 (3) 0
10 01 11 01 00 10 01 (3) 0
11 01 11 01 00 10 01 (3) 0
01 11 11 01 00 10 01 (3) 0
11 11 11 01 00 10 01 (3) 0
10 00 10 10 00 10 01 (3) 0
11 00 10 10 00 10 01 (3) 0
01 01 10 10 00 10 01 (3) 0
01 11 10 10 00 10 01 (3) 0
01 11 11 10 00 10 01 (3) 0
10 11 11 10 00 10 01 (3) 0
10 01 10 11 00 10 01 (3) 0
10 11 10 11 00 10 01 (3) 0
11 11 10 11 00 10 01 (3) 0
11 01 11 11 00 10 01 (3) 0
11 01 10 01 01 10 01 (3) 0
11 11 10 01 01 10 01 (3) 0
10 01 11 01 01 10 01 (3) 0
01 11 11 01 01 10 01 (3) 0
10 11 11 01 01 10 01 (3) 0
10 01 10 10 01 10 01 (3) 0
11 11 10 10 01 10 01 (3) 0
0

10 00 11 01 01 10 01 (2) 0
01 01 11 01 01 10 01 (2) 0
11 11 11 01 01 10 01 (2) 0
11 01 10 10 01 10 01 (2) 0
01 00 11 10 01 10 01 (2) 0
10 00 11 10 01 10 01 (2) 0
01 01 11 10 01 10 01 (2) 0
10 01 11 10 01 10 01 (2) 0
01 11 11 10 01 10 01 (2) 0
01 11 10 11 01 10 01 (2) 0
11 11 10 11 01 10 01 (2) 0
01 00 11 11 01 10 01 (2) 0
10 00 11 11 01 10 01 (2) 0
01 11 11 11 01 10 01 (2) 0
11 11 11 11 01 10 01 (2) 0
11 00 10 10 01 00 00 (2) 0
01 01 10 10 01 00 00 (2) 0
01 11 10 10 01 00 00 (2) 0
10 11 10 10 01 00 00 (2) 0
10 01 10 01 11 10 01 (2) 0
10 11 10 01 11 10 01 (2) 0
10 00 11 01 11 10 01 (2) 0
11 01 11 01 11 10 01 (2) 0
01 11 11 01 11 10 01 (2) 0
10 11 11 01 11 10 01 (2) 0
11 11 11 01 11 10 01 (2) 0
11 00 10 10 11 10 01 (2) 0
10 01 10 10 11 10 01 (2) 0
01 11 10 10 11 10 01 (2) 0
10 11 10 10 11 10 01 (2) 0
11 11 11 10 11 10 01 (2) 0
10 00 10 11 11 10 01 (2) 0
01 01 11 11 11 10 01 (2) 0
10 01 11 11 11 10 01 (2) 0
10 11 10 11 01 00 00 (2) 0
01 11 11 11 01 00 00 (2) 0
10 00 10 01 00 00 10 (2) 0
1

10 01 10 01 00 01 01 (1) 0
01 11 10 01 00 01 01 (1) 0
11 00 11 01 00 01 01 (1) 0
01 01 11 01 00 01 01 (1) 0
10 01 11 01 00 01 01 (1) 0
11 11 11 01 00 01 01 (1) 0
11 00 10 10 00 01 01 (1) 0
10 01 10 10 00 01 01 (1) 0
10 11 10 10 00 01 01 (1) 0
11 01 11 10 00 01 01 (1) 0
11 01 10 11 00 01 01 (1) 0
01 00 11 11 00 01 01 (1) 0
10 00 11 11 00 01 01 (1) 0
10 11 11 11 00 01 01 (1) 0
01 00 10 01 01 01 01 (1) 0
11 00 10 01 01 01 01 (1) 0
01 11 10 01 01 01 01 (1) 0
11 11 10 01 01 01 01 (1) 0
10 01 11 01 01 01 01 (1) 0
10 00 10 10 01 01 01 (1) 0
10 11 10 10 01 01 01 (1) 0
10 00 11 10 01 01 01 (1) 0
10 11 11 10 01 01 01 (1) 0
11 11 11 10 01 01 01 (1) 0
11 11 10 11 01 01 01 (1) 0
01 01 11 11 01 01 01 (1) 0
11 00 10 01 01 00 00 (1) 0
11 01 10 01 01 00 00 (1) 0
10 01 11 01 01 00 00 (1) 0
11 01 10 01 11 01 01 (1) 0
01 00 11 01 11 01 01 (1) 0
10 01 11 01 11 01 01 (1) 0
11 11 11 01 11 01 01 (1) 0
01 00 10 10 11 01 01 (1) 0
11 00 10 10 11 01 01 (1) 0
01 11 10 10 11 01 01 (1) 0
10 11 10 10 11 01 01 (1) 0
1